# Импорт данных из 

In [46]:
from lr2 import x_middle, disp, sko, S2, N, spaces, spaces_abs_freq
import math
import pandas as pd
import numpy as np

In [47]:
import scipy

# функция Лапласа
def laplas(x):
    return scipy.special.erf(x/2**0.5)/2 

In [48]:
S = sqrt(S2)
print(pd.DataFrame({
    "Интервал": map(lambda x: "[{},{})".format(x[0], x[1]), spaces),
    "Середина": map(lambda x: ((x[0] + x[1]) / 2).round(2), spaces),
    "Частоты": spaces_abs_freq
}))
print("N: {}".format(N))
print("Матожидание: {}".format(x_middle))
print("Дисперсия: {}".format(disp))
print("СКО: {}".format(sko))
print("Исправленная дисперсия: {}".format(S2))
print("Исправленное СКО: {}".format(S))

    Интервал  Середина  Частоты
0  [321,350)     335.5        2
1  [350,379)     364.5       10
2  [379,408)     393.5       10
3  [408,437)     422.5       13
4  [437,466)     451.5       36
5  [466,495)     480.5       19
6  [495,524)     509.5       13
7  [524,553)     538.5       11
N: 114
Матожидание: 453.280701754386
Дисперсия: 2541.9606802092953
СКО: 50.417860726227715
Исправленная дисперсия: 2564.4559074677845
Исправленное СКО: 50.640457220169175


In [49]:
_gamma = 0.95
_t = 1.980
_t = S * t / np.sqrt(N)
_i1 = x_middle - _t
_i2 = x_middle + _t
print("Для надежности {} и объема выборки n={} было выбрано значение t={} по приложению 6".format(_gamma, N, _t))
print("Доверительный интервал для матожидания: ({}, {})".format(_i1, _i2))

Для надежности 0.95 и объема выборки n=114 было выбрано значение t=9.3909684774352 по приложению 6
Доверительный интервал для матожидания: (443.8897332769508, 462.67167023182117)


In [50]:
_q = 0.143
_i1 = S * (1 - _q)
_i2 = S * (1 + _q)
print("Для надежности {} и объема выборки n={} было выбрано значение q={} по приложению 7".format(_gamma, N, _q))
print("Доверительный интервал для СКО: ({}, {})".format(_i1, _i2))

Для надежности 0.95 и объема выборки n=114 было выбрано значение q=0.143 по приложению 7
Доверительный интервал для СКО: (43.39887183768498, 57.88204260265337)


In [59]:
_x = list(map(lambda x: x[0], spaces))
_x.append(spaces[-1][1])
df = pd.DataFrame()
df['x_i'] = _x[:-1]
df['x_(i+1)'] = _x[1:]
df['n_i'] = spaces_abs_freq
df['z_i'] = df.iloc[:, 0].apply(lambda x: (x - x_middle) / S)
df['z_(i+1)'] = df.iloc[:, 1].apply(lambda x: (x - x_middle) / S)
df['F(z_i)'] = df.iloc[:, 3].apply(lambda x: laplas(x))
df['F(z_(i+1))'] = df.iloc[:, 4].apply(lambda x: laplas(x))
df['p_i'] = df.iloc[:, 5:7].apply(lambda x: x[1] - x[0], axis=1)
df['n_i1'] = df.iloc[:, 7].apply(lambda x: N * x)

df

,x_i,x_(i+1),n_i,z_i,z_(i+1),F(z_i),F(z_(i+1)),p_i,n_i1
0,321,350,2,-2.612155,-2.039490,-0.495501,-0.479299,0.016202,1.847017
1,350,379,10,-2.039490,-1.466825,-0.479299,-0.428788,0.050511,5.758279
2,379,408,10,-1.466825,-0.894161,-0.428788,-0.314382,0.114406,13.042305
3,408,437,13,-0.894161,-0.321496,-0.314382,-0.126083,0.188299,21.466121
4,437,466,36,-0.321496,0.251169,-0.126083,0.099158,0.225241,25.677459
5,466,495,19,0.251169,0.823833,0.099158,0.294983,0.195825,22.324017
6,495,524,13,0.823833,1.396498,0.294983,0.418718,0.123735,14.105771
7,524,553,11,1.396498,1.969163,0.418718,0.475533,0.056815,6.476919


In [67]:
df1 = pd.DataFrame()

df1['n_i'] = spaces_abs_freq
df1['n_i1'] = df['n_i1']
df1['(n_i - n_i1)^2 / n_i1'] = df1.iloc[:, :].apply(lambda x: (x[0] - x[1])**2 / x[1], axis=1)

X2_seen = df1.iloc[:,2].sum()

print("X2_наб = {}".format(X2_seen))
print("Число стееней свободы k = {} - 3 = {}".format(len(spaces), len(spaces)-3))
print("Для числа степеней свободы 5 X_крит = {}".format(11.07))
df1


X2_наб = 15.075911303606404
Число стееней свободы k = 8 - 3 = 5
Для числа степеней свободы 5 X_крит = 11.07


,n_i,n_i1,(n_i - n_i1)^2 / n_i1
0,2,1.847017,0.012671
1,10,5.758279,3.124579
2,10,13.042305,0.709661
3,13,21.466121,3.338992
4,36,25.677459,4.149743
5,19,22.324017,0.494942
6,13,14.105771,0.086683
7,11,6.476919,3.158640
